In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Optuna
import optuna
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO


# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt
import seaborn as sn

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [2]:
validacion = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

df = validacion[validacion['ESTACION'] == 4]

first_pred = df.FECHAHORA.min() + relativedelta(days = 2)

df = df[df['FECHAHORA'] >= first_pred]
df.reset_index(inplace = True, drop = True)



pred_xgb = pd.read_csv('datos/experimento_full/predicciones_validation_xgboost_station4_5_1_2.csv')
pred_xgb.reset_index(inplace = True, drop = True)

pred_svr = pd.read_csv('datos/experimento_full/predicciones_validation_svr_station4_5_1_2.csv')
pred_svr.reset_index(inplace = True, drop = True)


In [3]:
n = 5

new_index = pd.RangeIndex(len(pred_xgb)*(n+1))
pred_xgb_full = pd.DataFrame(np.nan, index=new_index, columns=pred_xgb.columns)
ids = np.arange(len(pred_xgb))*(n+1)
pred_xgb_full.loc[ids] = pred_xgb.values

In [4]:
pred_xgb_full.interpolate(inplace=True)

pred_xgb_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25920 entries, 0 to 25919
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  25920 non-null  float64
 1   TARGET      25920 non-null  float64
 2   FORECAST    25920 non-null  float64
 3   ESTACION    25920 non-null  float64
dtypes: float64(4)
memory usage: 810.1 KB


In [5]:
pred_xgb_full.rename(columns = {'FORECAST': 'FORECAST_XGB'}, inplace = True)
pred_svr.rename(columns = {'FORECAST': 'FORECAST_SVR'}, inplace = True)

p_xgb = pred_xgb_full.FORECAST_XGB
p_svr = pred_svr.FORECAST_SVR

df = pd.merge(p_xgb, df, right_index = True, left_index = True)
df = pd.merge(p_svr, df, right_index = True, left_index = True)

In [6]:
fig_val = go.Figure()

df_grafica = df

fig_val.add_trace(
    go.Scatter( y = list(df_grafica.FORECAST_XGB), x = list(df_grafica.FECHAHORA), name = 'Forecasts XGB', line=dict(color = 'Orange')))

fig_val.add_trace(
    go.Scatter( y = list(df_grafica.FORECAST_SVR), x = list(df_grafica.FECHAHORA), name = 'Forecasts SVR', line=dict(color = '#f03c3c')))

fig_val.add_trace(
    go.Scatter( y = list(df_grafica.AQI_MP2_5), x = list(df_grafica.FECHAHORA), name = 'Target', line=dict(color = '#0087e8')))

layout = go.Layout(
        title="Title",
        xaxis=dict(
            title="X Label"
        ),
        yaxis=dict(
            title="Y label"
        ) ) 

fig_val.update_layout( 
                        width = 600,
                        height = 700,
                        title = {
                                'text': "24hr prediction. XGBOOST vs SVR",
                                'y':0.9,
                                'x':0.5,
                                'xanchor': 'center',
                                'yanchor': 'top'}, 
                        
                        yaxis_title = 'AQI MP 2.5', 
                        xaxis_title = 'Date',
                        
                        legend= {
                                'xanchor': 'center',
                                'yanchor': 'bottom',
                                'y' : -0.7,
                                'x' : 0.0,
                        },
                        font=dict(
                                    size=16,
                                        ),
                        legend_title = 'References',
                        )

fig_val.write_html('graphs/modelos_finales_todas_estaciones/val_24h_station_4.html')

